In [ ]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy import interpolate
import glob
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


import heapq
from scipy.signal import argrelextrema

from scipy import signal
from scipy.ndimage.interpolation import shift
import pywt
from numpy.fft import fft
from numpy import zeros, floor, log10, log, mean, array, sqrt, vstack, cumsum, ones, log2, std
from numpy.linalg import svd, lstsq
import time

from sklearn.decomposition import PCA
import time
from joblib import Parallel, delayed
import multiprocessing

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from catboost import CatBoostRegressor, FeaturesData, Pool
from sklearn.metrics import mean_squared_error

In [ ]:
#==========================================================================================
#making time stamp uniform by Interpolation

def preprocess(data):
    freq=50
    ls=['X','Y','Z']
    t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],0.02)
    df=pd.DataFrame({'Timestamp':t1})
    for i in ls:
        fcubic = interpolate.interp1d(data.Timestamp, data[i], kind='cubic')
        df[i]=fcubic(t1)
    df.columns=['Timestamp','acc_X','acc_Y','acc_Z']
    return df

#==========================================================================================
#median filter
from scipy.signal import medfilt # import the median filter function
def median(signal):# input: numpy array 1D (one column)  
    #applying the median filter
    return  medfilt(np.array(signal), kernel_size=3) # applying the median filter order3(kernel_size=3)


#==========================================================================================
#components_selection_one_signal
import math # import math library


def components_selection_one_signal(t_signal):
    sampling_freq=50
    nyq=sampling_freq/float(2) # nyq is the nyquist frequency equal to the half of the sampling frequency[50/2= 25 Hz]

    freq1 = 0.3
    freq2 = 20

    t_signal=np.array(t_signal)
    t_signal_length=len(t_signal) # number of points in a t_signal
    
    # the t_signal in frequency domain after applying fft
    f_signal=np.fft.fft(t_signal) # 1D numpy array contains complex values (in C)
    
    # generate frequencies associated to f_signal complex values
    freqs=np.array(np.fft.fftfreq(t_signal_length, d=1/float(sampling_freq))) # frequency values between [-25hz:+25hz]
        
    df=pd.DataFrame({'freq':abs(freqs),'amplitute':f_signal})
    df['f_DC_signal']=np.where(df.freq>freq1,0,df.amplitute)
    df['f_noise_signal']=np.where(df.freq<=freq2,0,df.amplitute)
    df['f_body_signal']=np.where(df.freq<=freq1,0,np.where(df.freq>freq2,0,df.amplitute))

    
    # Inverse the transformation of signals in freq domain #
    # applying the inverse fft(ifft) to signals in freq domain and put them in float format
    t_DC_component= np.fft.ifft(np.array(df['f_DC_signal'])).real
    t_body_component= np.fft.ifft(np.array(df['f_body_signal'])).real
    t_noise=np.fft.ifft(np.array(df['f_noise_signal'])).real
    
    total_component=t_signal-t_noise # extracting the total component(filtered from noise) 
                                     #  by substracting noise from t_signal (the original signal).
    
    # return outputs mentioned earlier
    return (total_component,t_DC_component,t_body_component,t_noise) 


#=================================================================================================================
#Define verify gravity function
def mag_3_signals(df): # Euclidian magnitude
    return np.array(np.sqrt(np.square(df).sum(axis=1)))

def verify_gravity(data):
    
    acc_x=np.array(data['acc_X']) # copy acc_X column from dataframe in raw_dic having the key mentioned above
    acc_y=np.array(data['acc_Y'])# copy acc_Y column  from dataframe in raw_dic having the key mentioned above
    acc_z=np.array(data['acc_Z'])# copy acc_Z column  from dataframe in raw_dic having the key mentioned above

    # apply the filtering method to acc_[X,Y,Z] and store gravity components
    grav_acc_X=components_selection_one_signal(acc_x)[1] 
    grav_acc_Y=components_selection_one_signal(acc_y)[1]
    grav_acc_Z=components_selection_one_signal(acc_z)[1]
    
    # calculating gravity magnitude signal
    grav_acc_mag=mag_3_signals(grav_acc_X, grav_acc_Y,grav_acc_Z)
    print('mean value = ',round((sum(grav_acc_mag) / len(grav_acc_mag)),3),' g')
    
#=================================================================================================================    
#Define jerking and magnitude functions
def jerk_one_signal(signal):
    signal=pd.DataFrame(signal)
    jerk=(signal.shift(-1)-signal)/0.02
    return np.array(jerk.dropna()).transpose()[0]

def time_domain_signal(data):
    time_sig_df=pd.DataFrame()
    for column in ['acc_X','acc_Y','acc_Z']:
        t_signal=np.array(data[column])
        #med_filtred=median(t_signal)
        med_filtred=(t_signal)
        _,grav_acc,body_acc,_=components_selection_one_signal(med_filtred)
        body_acc_jerk=jerk_one_signal(body_acc)
        time_sig_df['t_body_'+column]=body_acc[:-1]
        time_sig_df['t_grav_'+column]= grav_acc[:-1]
        time_sig_df['t_body_acc_jerk_'+column[-1]]=body_acc_jerk

    # all 15 axial signals generated above are reordered to facilitate magnitudes signals generation
    new_columns_ordered=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                              't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                              't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z']


    # create new dataframe to order columns
    time_sig_df=time_sig_df[new_columns_ordered]

    # Magnitude Features
    for i in range(0,9,3):
        mag_col_name=new_columns_ordered[i][:-1]+'mag'# Create the magnitude column name related to each 3-axial signals
        time_sig_df[mag_col_name]=mag_3_signals(time_sig_df[new_columns_ordered[i:i+3]]) # store the signal_mag with its appropriate column name

    return(time_sig_df)

In [ ]:
#Fractal Dimension using Katz FD algorithm
def katz(data):
    n = len(data)-1
    L = np.hypot(np.diff(data), 1).sum() # Sum of distances
    d = np.hypot(data - data[0], np.arange(len(data))).max() # furthest distance from first point
    return np.log10(n) / (np.log10(d/L) + np.log10(n))




#Coefficient of variation
def coeff_var(a):
    output = np.std(a)/np.mean(a) #computing coefficient of variation
    return output

#Mean and variance of Vertex to Vertex Slope
def slope(x):
    
    amp_max = np.array(x[argrelextrema(np.array(x), np.greater)[0]])
    t_max = argrelextrema(np.array(x), np.greater)[0]
    amp_min = np.array(x[argrelextrema(np.array(x), np.less)[0]])
    t_min = argrelextrema(np.array(x), np.less)[0]
    t = np.concatenate((t_max,t_min),axis=0)
    t.sort()#sort on the basis of time

    amp = np.zeros(len(t))
    res = np.zeros(len(t))
    
    for l in range(len(t)):
        amp[l]=x[t[l]]

    amp_diff = first_diff(amp)
    t_diff = first_diff(t)

    for q in range(len(amp_diff)):
        res[q] = amp_diff[q]/t_diff[q]         
    
    res=res[~np.isnan(res)]
    return [np.mean(res),np.std(res)] #returning mean and std of vertex to vertex slope


#Hjorth Parameter
def hjorth(input):                                              
    hfeatures = []
    diff_input = np.diff(input)
    diff_diffinput = np.diff(diff_input)
    
    hjorth_activity = np.var(input)
    hjorth_mobility = np.sqrt(np.var(diff_input)/hjorth_activity)
    hjorth_diffmobility = np.sqrt(np.var(diff_diffinput)/np.var(diff_input))
    hjorth_complexity = hjorth_diffmobility/hjorth_mobility
    
    hfeatures.append(hjorth_activity)
    hfeatures.append(hjorth_mobility)
    hfeatures.append(hjorth_complexity)
    
    return hfeatures  #returning hjorth activity, hjorth mobility , hjorth complexity


#Kurtosis
def kurtosis(a):
    mean_i = np.mean(a) # Saving the mean of array i
    std_i = np.std(a) # Saving the standard deviation of array i
    t = 0.0
    for j in a:
        t += (pow((j-mean_i)/std_i,4)-3)
    kurtosis_i = t/len(a) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
    return kurtosis_i


#Second difference Mean,Max,std
def sec_diff(b):
    temp1 = abs(b-b.shift(1)) # Obtaining the 1st Diffs
    t = abs(temp1-temp1.shift(1)) # Summing the 2nd Diffs
    output = t.mean() # Calculating the mean of the 2nd Diffs
    return [t.mean(),t.max(),t.std()]

#Skewness
def skewness(a):
    import scipy.stats as sp
    skew_array=sp.stats.skew(a,axis=0,bias=True)
    return skew_array #returning skewness


#First Difference Mean,Max,std
def first_diff_mean(a):
    output = abs(a-a.shift(1)) # Obtaining the 1st Diffs
    return [output.mean(),output.max(),output.std()] #returns first diff mean,max,min

#First Difference
def first_diff(a):
    if str(type(a))!="<class 'pandas.core.series.Series'>":
        a=pd.DataFrame(a)[0]
    output = a-a.shift(1)# Obtaining the 1st Diffs
    return output


#wavelet features
def wavelet_features(epoch): 
    wfeatures = []
    
    #calculating the coefficients of wavelet transform.
    cA_values,cD_values=pywt.dwt(epoch,'coif1')
    
    cA_square=np.array([x for x in (np.square(cA_values)).tolist() if x != 0])
    cD_square=np.array([x for x in (np.square(cD_values)).tolist() if x != 0])
    
      
    wfeatures.append(np.mean(cA_values)) #cA_mean
    wfeatures.append(abs(np.std(cA_values))) #cA_std
    wfeatures.append(abs(np.sum(np.square(cA_values)))) #cA_Energy
    wfeatures.append(abs(np.sum(cA_square * np.log(cA_square)))) #Entropy_A
    
    wfeatures.append(np.mean(cD_values)) #cD_mean
    wfeatures.append(abs(np.std(cD_values))) #cD_std
    wfeatures.append(abs(np.sum(np.square(cD_values)))) #cD_Energy
    wfeatures.append(abs(np.sum(cD_square * np.log(cD_square)))) #Entropy_D

    return wfeatures # returning 'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Approximate Entropy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Detailed Entropy'


In [ ]:
def features_generation(t_window):
    
    # select mag columns : the last 3 columns in a time domain window
    
    mag_columns=t_window.columns # mag columns' names
    
    t_mag_features=[] # a global list will contain all time domain magnitude features
    
    for col in mag_columns: # iterate throw each mag column
        
        fkatz = [katz(t_window[col])] # 1 value
        fcoeff_var   = [coeff_var(t_window[col])] # 1 value
        fslope    = slope(t_window[col])# 2 value
        fhjorth    = hjorth(t_window[col])# 3 value
        fsec_diff    = sec_diff(t_window[col])# 3 value
        ffirst_diff_mean = first_diff_mean(t_window[col])# 3 value
        fwavelet_features    = wavelet_features(t_window[col])# 8 value
        
        # 13 value per each t_mag_column
        col_mag_values = fkatz+fcoeff_var+fslope+fhjorth+fsec_diff+ffirst_diff_mean+fwavelet_features
        
        # col_mag_values will be added to the global list
        t_mag_features= t_mag_features+ col_mag_values
    
    # t_mag_features contains 65 values = 13 values (per each t_mag_column) x 5 (t_mag_columns)
    return t_mag_features
 
def Dataset_Generation_PipeLine(b):
    data=preprocess(pd.read_csv(b))
    time_sig_df=time_domain_signal(data)
    # concatenate all features and append the activity id and the user id
    row= features_generation(time_sig_df)
    return(row)


In [ ]:
def features_names():
    # Generating time feature names
    
    # time domain magnitude signals' names
    magnitude_signals=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                       't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                       't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z',
                       't_body_acc_Mag','t_grav_acc_Mag','t_body_acc_jerk_Mag']

    # functions' names:
    t_one_input_features_name1=['_katz()','_coeff_var()']

    t_one_input_features_slope=['_slope_mean()','_slope_std()']

    t_one_input_features_hjorth=['_hjorth_activity()','_hjorth_mobility()','_hjorth_complexity()']

    t_one_input_features_sec_diff=['_sec_diff_mean()','_sec_diff_max()','_sec_diff_std()']
    t_one_input_features_first_diff=['_first_diff_mean()','_first_diff_max()','_first_diff_std()']
    
    t_one_input_features_wavelet=['_wavelet_cA_mean()','_wavelet_cA_std()','_wavelet_cA_Energy()','_wavelet_Entropy_A()',
                                  '_wavelet_cD_mean()','_wavelet_cD_std()','_wavelet_cD_Energy()','_wavelet_Entropy_D()']
    
    features=[]# Empty list : it will contain all time domain features' names
    
    for columns in magnitude_signals: # iterate throw time domain magnitude column names

        # build feature names related to that column
        #list 1
        for feature in t_one_input_features_name1:
            newcolumn=columns+feature
            features.append(newcolumn)
        
        # list 2
        for feature in t_one_input_features_slope: 
            newcolumn=columns+feature
            features.append(newcolumn)
            
       
        # list 3
        for feature in t_one_input_features_hjorth:
            newcolumn=columns+feature
            features.append(newcolumn)
            
        # list 4
        for feature in t_one_input_features_sec_diff:
            newcolumn=columns+feature
            features.append(newcolumn)
        # list 5
        for feature in t_one_input_features_first_diff:
            newcolumn=columns+feature
            features.append(newcolumn)
        # list 6
        for feature in t_one_input_features_wavelet:
            newcolumn=columns+feature
            features.append(newcolumn)
            
        
    ###########################################################################################################
    time_list_features=features
    
    return time_list_features # return all time domain features' names

In [ ]:
from joblib import Parallel, delayed
import multiprocessing
import time

In [ ]:
# training
a=glob.glob("training_data/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in a)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#47.693 min 

In [ ]:
training_data=pd.DataFrame(result)
training_data.columns=features_names()
training_data['measurement_id']=[item[len('training_data/'):-4] for item in a]
print(training_data.shape) #(1858, 253)
training_data.head()

In [ ]:
#ancillary
b=glob.glob("ancillary_data/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in b)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#12.124 Mins

In [ ]:
ancillary_data=pd.DataFrame(result)
ancillary_data.columns=features_names()
ancillary_data['measurement_id']=[item[len('ancillary_data/'):-4] for item in b]
print(ancillary_data.shape)
ancillary_data.head()

In [ ]:
#merging training and ancillary_data
data=training_data.append(pd.DataFrame(ancillary_data),ignore_index=True)
print(data.shape)
data.head()

In [ ]:
#export part3 features of training data for cispd
data.to_csv('cispd_comp_training_abhiroop_lastfeatures.csv',index=False)

In [ ]:
# testing
a=glob.glob("testing_data/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in a)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#18.773 min 

testing_data=pd.DataFrame(result)
testing_data.columns=features_names()
testing_data['measurement_id']=[item[len('testing_data/'):-4] for item in a]
print(testing_data.shape) #(1858, 253)
testing_data.head()

In [ ]:
#export part3 features of testing data for cispd
testing_data.to_csv('cispd_comp_testing_abhiroop_lastfeatures.csv',index=False)

# Clinical Data

In [ ]:
#CIS-PD_Demographics
cis_pd_demo=pd.read_csv('clinical_data/CIS-PD_Demographics.csv')
cis_pd_demo=cis_pd_demo.drop(['Race','Ethnicity'],axis=1)
cis_pd_demo['Gender']=cis_pd_demo['Gender'].map({'Male':1,'Female':0})
print(cis_pd_demo.shape)
cis_pd_demo.head()

In [ ]:
#CIS-PD_UPDRS_Part1_2_4
cis_pd_updrs_part1_2_4=pd.read_csv('clinical_data/CIS-PD_UPDRS_Part1_2_4.csv')
cis_pd_updrs_part1_2_4.drop('Visit',axis=1,inplace=True)
print(cis_pd_updrs_part1_2_4.shape)
cis_pd_updrs_part1_2_4.head()

In [ ]:
#CIS-PD_UPDRS_Part3
cis_pd_updrs_part3=pd.read_csv('clinical_data/CIS-PD_UPDRS_Part3.csv')
cis_pd_updrs_part3.drop(['UPDRS_3a','UPDRS_3b','UPDRS_3c','UPDRS_3C1','ParticipantState','UPDRS_3.19B'],axis=1,inplace=True)

#Cleaning and feature creation. from CIS-PD_UPDRS_Part3
visit1=cis_pd_updrs_part3.Visit.map({'Baseline':0,'2 Weeks: Time 0':1,'2 Weeks: Time 60':1})
visit2=cis_pd_updrs_part3.Visit.map({'Baseline':0,'2 Weeks: Time 0':0,'2 Weeks: Time 60':1})

col=[e for e in cis_pd_updrs_part3.columns if e not in ('subject_id','Visit')]
cis_pd_updrs_part3=cis_pd_updrs_part3.groupby(['subject_id'])[col].ffill().bfill()

cis_pd_updrs_part3['Visit']=visit2
cis_pd_updrs_part3['Visit_Type']=visit1
cis_pd_updrs_part3['UPDRS_3.19A']=cis_pd_updrs_part3['UPDRS_3.19A'].map({'Yes':1,'No':0})

cis_pd_updrs_part3['Rigidity']=cis_pd_updrs_part3[['UPDRS_3.3 Neck','UPDRS_3.3 Right Upper Extremity','UPDRS_3.3 Left Upper Extremity','UPDRS_3.3 Right Lower Extremity','UPDRS_3.3 Left Lower Extremity']].mean(axis=1)
cis_pd_updrs_part3['Tapping']=cis_pd_updrs_part3[['UPDRS_3.4 Right Hand','UPDRS_3.4 Left Hand','UPDRS_3.7 Right Foot','UPDRS_3.7 Left Foot']].mean(axis=1)
cis_pd_updrs_part3['Body Tremor']=cis_pd_updrs_part3[['UPDRS_3.15 Right Hand','UPDRS_3.15 Left Hand','UPDRS_3.16 Right Hand','UPDRS_3.16 Left Hand','UPDRS_3.17 Right Upper Extremity','UPDRS_3.17 Left Upper Extremity','UPDRS_3.17 Right Lower Extremity','UPDRS_3.17 Left Lower Extremity','UPDRS_3.17 Lip-Jaw']].mean(axis=1)
cis_pd_updrs_part3['Movement']=cis_pd_updrs_part3[['UPDRS_3.5 Right Hand','UPDRS_3.5 Left Hand','UPDRS_3.6 Right Hand','UPDRS_3.6 Left Hand']].mean(axis=1)

type1=cis_pd_updrs_part3.copy()
type1.drop(['Visit','Visit_Type'],axis=1,inplace=True)
numeric_cols = list(set(type1.columns) - set(['subject_id']))
type1[numeric_cols] += 1

type2=type1.groupby('subject_id').pct_change()
type2['subject_id']=cis_pd_updrs_part3['subject_id']
type2=type2.groupby('subject_id').max().reset_index().fillna(0)
type2.columns=['subject_id']+list('PerChange_'+type2.columns[1:])
cis_pd_updrs_part3=pd.merge(cis_pd_updrs_part3[cis_pd_updrs_part3.Visit==0].drop(['Visit'],axis=1),type2,on='subject_id')
print(cis_pd_updrs_part3.shape)
cis_pd_updrs_part3.head()


In [ ]:
#combining all clinical data
clinical_data=pd.merge(cis_pd_demo,cis_pd_updrs_part1_2_4,on='subject_id')
clinical_data=pd.merge(clinical_data,cis_pd_updrs_part3,on='subject_id')
print(clinical_data.shape)
clinical_data.head()

In [ ]:
#exporting clinical data
clinical_data.to_csv('cispd_clinical_preprocessed.csv',index=False)